In [1]:
import json
import os
from dataset import read_jsonl
import re
from transformers import AutoTokenizer


os.chdir('/home/sangmin/grade-school-math/grade_school_math/')

/home/sangmin/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


data load

In [2]:
path = os.path.join("data/train.jsonl")
objs = read_jsonl(path)

In [3]:
rawdata = []
for i in objs:
    # for key in i:
    raw = {}
    r = i['answer']
    t = re.findall('.+\n####', r)
    s = re.findall('\d+$', r)
    raw['context'] = t[0]
    raw['answers'] = s[0]
    raw['question'] = i['question']
        
    rawdata.append(raw)
#{'question': 'Nina enjoys keeping insects as pets. She has 3 spiders and 50 ants. Each spider has 8 eyes. Each ant has 2 eyes. How many eyes are there in total among Nina’s pet insects?', 
#'answer': 'The number of eyes from the spiders is 3 * 8 = <<3*8=24>>24 eyes\nThe number of eyes from the ants is 50 * 2 = <<50*2=100>>100 eyes\nThe total number of eyes among Nina’s insects is 24 + 100 = <<24+100=124>>124 eyes\n#### 124'}


In [4]:
path = os.path.join("data/test.jsonl")
objs = read_jsonl(path)

In [5]:
val = []
for i in objs:
    # for key in i:
    raw = {}
    r = i['answer']
    t = re.findall('.+\n####', r)
    s = re.findall('\d+$', r)
    raw['context'] = t[0]
    raw['answers'] = s[0]
    raw['question'] = i['question']
        
    rawdata.append(val)
#{'question': 'Nina enjoys keeping insects as pets. She has 3 spiders and 50 ants. Each spider has 8 eyes. Each ant has 2 eyes. How many eyes are there in total among Nina’s pet insects?', 
#'answer': 'The number of eyes from the spiders is 3 * 8 = <<3*8=24>>24 eyes\nThe number of eyes from the ants is 50 * 2 = <<50*2=100>>100 eyes\nThe total number of eyes among Nina’s insects is 24 + 100 = <<24+100=124>>124 eyes\n#### 124'}


In [6]:
a = []
for i in rawdata:
    for key in i: 
        c = len(i['context'])
        a.append(c)

print('the longest length for context is {}'.format(max(a)))
print('the shortest length for context is {}'.format(min(a)))

the longest length for context is 321
the shortest length for context is 9


tokenizer

In [7]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
# context = rawdata[1]["context"]
# question = rawdata[1]["question"]

# inputs = tokenizer(question, context)
# tokenizer.decode(inputs["input_ids"])

'[CLS] Weng earns $ 12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? [SEP] Working 50 minutes, she earned 0. 2 x 50 = $ < < 0. 2 * 50 = 10 > > 10. # # # # [SEP]'

In [9]:
# inputs = tokenizer(
#     question,
#     context,
#     max_length=100,
#     truncation="only_second",
#     stride=50, #assertion failed: stride < max_len when rawdata[10]
#     return_overflowing_tokens=True,
#     )

# inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])

preprocess_training_examples

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = rawdata.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=rawdata.column_names,
)
len(rawdata), len(train_dataset)

preprocess_validation_examples

In [2]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

AttributeError: 'str' object has no attribute 'print'

In [ ]:
train_dataset = val.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=val.column_names,
)
len(val), len(train_dataset)

Fine-tuning the model with Keras

In [ ]:
small_eval_set = val.select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val.column_names,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})